In [121]:
import pandas as pd
import numpy as np
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("https://raw.githubusercontent.com/nrharbeck/EnergyPredictionML/main/DataProcessing/EIA_DSIRE_Data_Tech.csv")

df

,Unnamed: 0,Index,Date,Generation,Copyright,description,end,f,geography,iso3166,name,series_id,source,start,units,State,Solar - Passive_Financial Incentive,Solar - Passive_Regulatory Policy,Solar Water Heat_Financial Incentive,Solar Water Heat_Regulatory Policy,Solar Space Heat_Financial Incentive,Solar Space Heat_Regulatory Policy,Geothermal Electric_Financial Incentive,Geothermal Electric_Regulatory Policy,Solar Thermal Electric_Financial Incentive,Solar Thermal Electric_Regulatory Policy,Solar Thermal Process Heat_Financial Incentive,Solar Thermal Process Heat_Regulatory Policy,Solar Photovoltaics_Financial Incentive,Solar Photovoltaics_Regulatory Policy,Wind (All)_Financial Incentive,Wind (All)_Regulatory Policy,Biomass_Financial Incentive,Biomass_Regulatory Policy,Hydroelectric_Financial Incentive,Hydroelectric_Regulatory Policy,Hydrogen_Financial Incentive,Hydrogen_Regulatory Policy,Geothermal Heat Pumps_Financial Incentive,Geothermal Heat Pumps_Regulatory Policy,...,Other Distributed Generation Technologies_Financial Incentive,Other Distributed Generation Technologies_Regulatory Policy,Vending Machine Controls_Financial Incentive,Vending Machine Controls_Regulatory Policy,Commercial Cooking Equipment_Financial Incentive,Commercial Cooking Equipment_Regulatory Policy,Personal Computing Equipment_Financial Incentive,Personal Computing Equipment_Regulatory Policy,Data Center Equipment_Financial Incentive,Data Center Equipment_Regulatory Policy,Microturbines_Financial Incentive,Microturbines_Regulatory Policy,Reflective Roofs_Financial Incentive,Reflective Roofs_Regulatory Policy,Pool Pumps_Financial Incentive,Pool Pumps_Regulatory Policy,LED Lighting_Financial Incentive,LED Lighting_Regulatory Policy,Tankless Water Heater_Financial Incentive,Tankless Water Heater_Regulatory Policy,Commercial Refrigeration Equipment_Financial Incentive,Commercial Refrigeration Equipment_Regulatory Policy,Industrial Systems General_Financial Incentive,Industrial Systems General_Regulatory Policy,Industrial System / Process Specific_Financial Incentive,Industrial System / Process Specific_Regulatory Policy,Building Systems_Financial Incentive,Building Systems_Regulatory Policy,HVAC_Financial Incentive,HVAC_Regulatory Policy,Load Management_Financial Incentive,Load Management_Regulatory Policy,Energy Sources_Financial Incentive,Energy Sources_Regulatory Policy,Lithium-ion_Financial Incentive,Lithium-ion_Regulatory Policy,Offshore Wind_Financial Incentive,Offshore Wind_Regulatory Policy,No Programs Available_Financial Incentive,No Programs Available_Regulatory Policy
0,0,0,2017-01-01,50.987891,None,See http://www.eia.gov/environment/emissions/s...,2017,A,USA-AL,USA-AL,"Electric power carbon dioxide emissions, all f...",EMISS.CO2-TOTV-EC-TO-AL.A,"EIA, U.S. Energy Information Administration",1980,million metric tons CO2,Alabama,1,0,3,0,0,0,2,1,0,1,0,0,22,1,0,0,18,2,13,2,0,1,13,0,...,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,2016-01-01,55.619153,None,See http://www.eia.gov/environment/emissions/s...,2017,A,USA-AL,USA-AL,"Electric power carbon dioxide emissions, all f...",EMISS.CO2-TOTV-EC-TO-AL.A,"EIA, U.S. Energy Information Administration",1980,million metric tons CO2,Alabama,1,0,2,0,0,0,2,1,0,1,0,0,20,1,0,0,16,1,12,2,0,1,13,0,...,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2,2015-01-01,62.391808,None,See http://www.eia.gov/environment/emissions/s...,2017,A,USA-AL,USA-AL,"Electric power carbon dioxide emissions, all f...",EMISS.CO2-TOTV-EC-TO-AL.A,"EIA, U.S. Energy Information Administration",1980,million metric tons CO2,Alabama,1,0,2,0,0,0,2,1,0,1,0,0,17,1,0,0,16,1,12,2,0,1,12,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,3,2014-01-01,65.591036,None,See http://www.eia.gov/environment/emissions/s...,2017,A,USA-AL,USA-AL,"Electric power carbon dioxide emissions, all f...",EMISS.CO2-TOTV-EC-TO-AL.A,"EIA, U.S. Energy Infor

In [122]:
#Drop any NA rows for model building
df = df.dropna(how='any')

#Now remove

#Remove CO2 emissions
df = df[~df.series_id.str.contains("EMISS.CO2-TOTV")]

#Make a new column with the EIA generation categories
df['generation_energy'] = df['series_id'].str[9:-8]

#Guide to see if strings changse from https://stackoverflow.com/questions/40348541/pandas-diff-with-string
df['Series_Change'] = df['series_id'].ne(df['series_id'].shift().bfill()).astype(int)
df["Generation_Diff"] = np.where(-df['Generation'].diff() > 0, 1, 0) 
df['Generation_Increase'] = np.where((df["Generation_Diff"] > 0) & (df['Series_Change'] == 0), 1, 0)

#Split data into features and target. Descriptive EIA data is removed here.
X = df.drop(columns=['Generation', 'Series_Change', 'Generation_Diff', 'Generation_Increase', 'Unnamed: 0', 'Index', 'units', 'Copyright', 'description','end','f','geography','iso3166','name','source','start', 'series_id'])
y = df['Generation_Increase']

#Encode categorical features
#print(X.columns)
X = pd.get_dummies(X, columns=['Date','generation_energy','State'])


In [123]:
#Split data into train and test sets. Validation with the training set will be incorporated into the pipeline below
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#print(X_test[:6])
#print(y_test[:6])


In [ ]:
#Compare Performance with a Logistic Regression model
#Guide from Professor Yuxiao Huang, The George Washinton University
from sklearn.preprocessing import LabelEncoder
#Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

# Implement me
le = LabelEncoder()
y_train = le.fit_transform(y_train)

print(pd.DataFrame(data=y_train, columns=['Generation_Increase'])['Generation_Increase'].value_counts())

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

clfs = {'lr': LogisticRegression(random_state=0),
        'mlp': MLPClassifier(random_state=0),
        'dt': DecisionTreeClassifier(random_state=0),
        'rf': RandomForestClassifier(random_state=0),
        'xgb': XGBClassifier(seed=0)}

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe_clfs = {}

for name, clf in clfs.items():
    pipe_clfs[name] = Pipeline([('StandardScaler', StandardScaler()),('clf', clf)])

param_grids = {}
C_range = [10 ** i for i in range(-4, 5)]

#Logistic Regression Parameter Grid
param_grid = [{'clf__multi_class': ['ovr'], 
               'clf__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
               'clf__C': C_range},
              
              {'clf__multi_class': ['multinomial'],
               'clf__solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
               'clf__C': C_range}]

param_grids['lr'] = param_grid

#MLP Parameter Grid
param_grid = [{'clf__hidden_layer_sizes': [10, 100],
               'clf__activation': ['identity', 'logistic', 'tanh', 'relu']}]
param_grids['mlp'] = param_grid

#Decision Tree Parameter Grid
param_grid = [{'clf__min_samples_split': [2, 10, 30],
               'clf__min_samples_leaf': [1, 10, 30]}]
param_grids['dt'] = param_grid

#Random Forest Parameter Grid
param_grid = [{'clf__n_estimators': [10, 100, 1000],
               'clf__min_samples_split': [2, 10, 30],
               'clf__min_samples_leaf': [1, 10, 30]}]
param_grids['rf'] = param_grid

#XGBoost Parameter Grid
param_grid = [{'clf__eta': [10 ** i for i in range(-4, 1)],
               'clf__gamma': [0, 10, 100],
               'clf__lambda': [10 ** i for i in range(-4, 5)]}]
param_grids['xgb'] = param_grid

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# The list of [best_score_, best_params_, best_estimator_]
best_score_param_estimators = []

# For each classifier
for name in pipe_clfs.keys():
    # GridSearchCV
    gs = GridSearchCV(estimator=pipe_clfs[name],
                      param_grid=param_grids[name],
                      scoring='accuracy',
                      n_jobs=1,
                      iid=False,
                      cv=StratifiedKFold(n_splits=10,
                                         shuffle=True,
                                         random_state=0))
    # Fit the pipeline
    gs = gs.fit(X_train, y_train)
    
    # Update best_score_param_estimators
    best_score_param_estimators.append([gs.best_score_, gs.best_params_, gs.best_estimator_])

# Sort best_score_param_estimators in descending order of the best_score_
best_score_param_estimators = sorted(best_score_param_estimators, key=lambda x : x[0], reverse=True)

# For each [best_score_, best_params_, best_estimator_]
for best_score_param_estimator in best_score_param_estimators:
    # Print out [best_score_, best_params_, best_estimator_], where best_estimator_ is a pipeline
    # Since we only print out the type of classifier of the pipeline
    print([best_score_param_estimator[0], best_score_param_estimator[1], type(best_score_param_estimator[2].named_steps['clf'])], end='\n\n')

1    5017
0    4334
Name: Generation_Increase, dtype: int64
[0.6622822112527995, {'clf__eta': 0.0001, 'clf__gamma': 0, 'clf__lambda': 0.0001}, <class 'xgboost.sklearn.XGBClassifier'>]

[0.6585361533890947, {'clf__activation': 'logistic', 'clf__hidden_layer_sizes': 100}, <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>]

[0.658110288404406, {'clf__C': 0.1, 'clf__multi_class': 'ovr', 'clf__solver': 'saga'}, <class 'sklearn.linear_model._logistic.LogisticRegression'>]

[0.6358670414552767, {'clf__min_samples_leaf': 30, 'clf__min_samples_split': 2, 'clf__n_estimators': 1000}, <class 'sklearn.ensemble._forest.RandomForestClassifier'>]

[0.6349052744640981, {'clf__min_samples_leaf': 30, 'clf__min_samples_split': 2}, <class 'sklearn.tree._classes.DecisionTreeClassifier'>]



In [ ]:
y_pred = best_score_param_estimators[1][2].predict(X_test)
print(y_pred)
print(np.array(y_test))
print("Accuracy on the test set:", round((1-(np.abs(y_pred - y_test).sum()/len(y_pred)))*100,8),"percent")

[1 0 0 ... 1 0 1]
[0 0 0 ... 1 0 1]
Accuracy on the test set: 65.78272027 percent


In [113]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import KFold
# determine the number of input features
n_features = X_train.shape[1]

#Set up cross validation with guide from https://www.machinecurve.com/index.php/2020/02/18/how-to-use-k-fold-cross-validation-with-keras/
# Define the K-fold Cross Validator
kfold = KFold(n_splits=3, shuffle=True)
acc_per_fold =[]
loss_per_fold = []
# K-fold Cross Validation model evaluation
fold_no = 1
"""
for train, val in kfold.split(X_train, y_train):

  # define model
  hidden_layers = 3
  model = Sequential()
  model.add(Dense(100, activation='sigmoid', kernel_initializer='he_normal', input_shape=(n_features,)))
  for layer in range(hidden_layers):
    model.add(Dense(30, activation='relu', kernel_initializer='he_normal'))
  model.add(Dense(1, activation='sigmoid'))

  # compile the model
  model.compile(optimizer='adam', loss='BinaryCrossentropy', metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(X_train.iloc[train], y_train.iloc[train],
              batch_size=32,
              epochs=150,
              verbose=0)

  # Generate generalization metrics
  scores = model.evaluate(X_train.iloc[val], y_train.iloc[val], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1
"""  
# define model without cross validation
hidden_layers = 3
model = Sequential()
model.add(Dense(100, activation='sigmoid', kernel_initializer='he_normal', input_shape=(n_features,)))
for layer in range(hidden_layers):
  model.add(Dense(30, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='BinaryCrossentropy', metrics=['accuracy'])
# fit the model
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=0)
# Generate generalization metrics
scores = model.evaluate(X_test, y_test, verbose=0)

In [115]:
# evaluate the model
error_train = model.evaluate(X_train, y_train, verbose=0)
error = model.evaluate(X_test, y_test, verbose=0)

print('Training set accuracy: %.3f percent' % (error_train[1]*100))
print('Test set accuracy: %.3f percent' % (error[1]*100))

# make a prediction
#print("Accuracy on the train set:", round((1-(np.abs(((yhat>0.50)*1).reshape(1,-1) - np.array(y_train).reshape(1,-1)).sum()/len(y_train)))*100,8),"percent")

Training set accuracy: 79.617 percent
Test set accuracy: 62.960 percent


In [116]:
#Set up 2020 training sets
df = pd.read_csv("https://raw.githubusercontent.com/nrharbeck/EnergyPredictionML/main/DataProcessing/EIA_DSIRE_Data_Tech.csv")

#Drop any NA rows for model building
df = df.dropna(how='any')

#Remove CO2 emissions
df = df[~df.series_id.str.contains("EMISS.CO2-TOTV")]

#Make a new column with the EIA generation categories
df['generation_energy'] = df['series_id'].str[9:-8]
all_dates = df['Date'].unique()


#Select only most recent data for real time predictions
df2 = df.groupby(['generation_energy','State'],as_index=False).agg({'Date':'max'})

for date in range(len(all_dates)):
    df2 = df2.append(pd.Series(0, index=df2.columns), ignore_index=True)
    df2['Date'].iloc[-1] = all_dates[date]

for date in range(len(all_dates)):
    df = df.append(pd.Series(0, index=df.columns), ignore_index=True)
    df['Date'].iloc[-1] = all_dates[date]

df = df.merge(df2, on=['Date','generation_energy', 'State'])

#Guide to see if strings changse from https://stackoverflow.com/questions/40348541/pandas-diff-with-string
df['Series_Change'] = df['series_id'].ne(df['series_id'].shift().bfill()).astype(int)
df["Generation_Diff"] = np.where(-df['Generation'].diff() > 0, 1, 0) 
df['Generation_Increase'] = np.where((df["Generation_Diff"] > 0) & (df['Series_Change'] == 0), 1, 0)

#Split data into features and target. Descriptive EIA data is removed here.
X = df.drop(columns=['Generation', 'Series_Change', 'Generation_Diff', 'Generation_Increase', 'Unnamed: 0', 'Index', 'units', 'Copyright', 'description','end','f','geography','iso3166','name','source','start', 'series_id'])
y = df['Generation_Increase']

#Encode categorical features
#print(X.columns)
X = pd.get_dummies(X, columns=['Date','generation_energy','State'])
#Remove rows with all zeros from columns
X = X.drop(columns='generation_energy_0')
X = X.drop(columns='State_0')


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [117]:
X[X.filter(like="Financial Incentive").columns] += 1
yhat = model.predict(X)
X[X.filter(like="Financial Incentive").columns] -= 1


X[X.filter(like="Regulatory Policy").columns] += 1
yhat2 = model.predict(X)
X[X.filter(like="Regulatory Policy").columns] -= 1

X['PredictedFI'] = yhat
X['PredictedRP'] = yhat2
X['DifferenceFI-RP'] = X['PredictedRP'] - X['PredictedFI']
print(X[['PredictedFI','PredictedRP']])

print(X['DifferenceFI-RP'])

      PredictedFI  PredictedRP
0    1.030773e-03     0.000033
1    4.900189e-05     0.000010
2    1.979470e-04     0.000031
3    9.459049e-03     0.213684
4    4.784048e-03     0.000033
..            ...          ...
758  1.212885e-10     0.999999
759  1.064682e-13     0.995117
760  7.049885e-10     1.000000
761  2.687623e-11     1.000000
762  1.917553e-16     0.915606

[763 rows x 2 columns]
0     -0.000998
1     -0.000039
2     -0.000167
3      0.204225
4     -0.004751
         ...   
758    0.999999
759    0.995117
760    1.000000
761    1.000000
762    0.915606
Name: DifferenceFI-RP, Length: 763, dtype: float32


In [118]:
state_list = ['Alabama', 'Alaska', 'Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan', 'Minnesota', 'Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey', 'New Mexico','New York','North Carolina','North Dakota','Ohio', 'Oklahoma','Oregon','Pennsylvania','Rhode Island','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']
for state in state_list:
  if np.mean(X['DifferenceFI-RP'][(X[str('State_'+state)] == 1)]) < 0:
    significance = 'Regulatory Policy'
  else:
    significance = 'Financial Incentive'
  print(str(state+":"),significance)


Alabama: Financial Incentive
Alaska: Financial Incentive
Arizona: Financial Incentive
Arkansas: Financial Incentive
California: Financial Incentive
Colorado: Financial Incentive
Connecticut: Financial Incentive
Delaware: Financial Incentive
Florida: Financial Incentive
Georgia: Financial Incentive
Hawaii: Regulatory Policy
Idaho: Financial Incentive
Illinois: Financial Incentive
Indiana: Financial Incentive
Iowa: Financial Incentive
Kansas: Financial Incentive
Kentucky: Financial Incentive
Louisiana: Regulatory Policy
Maine: Financial Incentive
Maryland: Financial Incentive
Massachusetts: Financial Incentive
Michigan: Regulatory Policy
Minnesota: Financial Incentive
Mississippi: Financial Incentive
Missouri: Financial Incentive
Montana: Financial Incentive
Nebraska: Financial Incentive
Nevada: Financial Incentive
New Hampshire: Regulatory Policy
New Jersey: Financial Incentive
New Mexico: Financial Incentive
New York: Regulatory Policy
North Carolina: Financial Incentive
North Dakota: 